In [15]:
# Import necessary libraries

import openai

In [17]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with API key
pc = Pinecone(
    api_key="pcsk_4U4srV_K3R6NSeBufDQUGDPN8U6P5vRQVtbdPuyPE8yDbw5KXmXmYqQC8RkeEUG7wVDbJN"  # Replace with your actual Pinecone API key
)

# Check if the index already exists
if "experiments" not in pc.list_indexes().names():
    # Create the index if it doesn't exist
    pc.create_index(
        name="experiments",
        dimension=1536,
        metric="cosine",  # Similarity metric: "cosine", "dotproduct", or "euclidean"
        spec=ServerlessSpec(
            cloud="aws",  # Choose cloud provider ("aws" or "gcp")
            region="us-east-1"  # Replace with your environment region
        )
    )

print("Index created successfully.")


Index created successfully.


In [18]:
index=pc.Index("experiments")

In [19]:
# Function to generate embeddings using OpenAI
from langchain.embeddings import OpenAIEmbeddings
def generate_embedding(text):
    embeddings  =  OpenAIEmbeddings(openai_api_key=openai_api_key,model="text-embedding-ada-002")
    embedding_response = embeddings.embed_query(text)
    return embedding_response

In [20]:
# Define schema metadata from your input data
schemas = [
    {
        "table": "customers",
        "description": "Stores customer information",
        "columns": [
            "customerNumber", "customerName", "contactLastName", "contactFirstName",
            "phone", "addressLine1", "addressLine2", "city", "state", "postalCode",
            "country", "salesRepEmployeeNumber", "creditLimit"
        ],
        "primary_key": ["customerNumber"],
        "foreign_keys": {"salesRepEmployeeNumber": "employees.employeeNumber"}
    },
    {
        "table": "employees",
        "description": "Stores employee information",
        "columns": [
            "employeeNumber", "lastName", "firstName", "extension", "email",
            "officeCode", "reportsTo", "jobTitle"
        ],
        "primary_key": ["employeeNumber"],
        "foreign_keys": {"reportsTo": "employees.employeeNumber"}
    },
    {
        "table": "offices",
        "description": "Stores office location information",
        "columns": [
            "officeCode", "city", "phone", "addressLine1", "addressLine2",
            "state", "country", "postalCode", "territory"
        ],
        "primary_key": ["officeCode"],
        "foreign_keys": {}
    },
    # Add remaining tables here...
]

In [21]:
import json

# Store schema metadata and embeddings in Pinecone
for schema in schemas:
    # Convert complex fields like `foreign_keys` to strings
    foreign_keys = schema["foreign_keys"]
    if not isinstance(foreign_keys, (str, list)):
        # Convert dictionary or complex object to JSON string
        foreign_keys = json.dumps(foreign_keys)

    metadata = {
        "table": schema["table"],
        "description": schema["description"],
        "columns": schema["columns"],
        "primary_key": schema["primary_key"],
        "foreign_keys": foreign_keys,  # Ensure this is now a string or valid type
    }
    embedding = generate_embedding(schema["description"])
    index.upsert([(schema["table"], embedding, metadata)])


In [22]:
def query_relevant_documents(user_query: str, index, top_k: int = 5):
    """
    Query Pinecone to retrieve relevant documents for a user's query.

    Args:
        user_query (str): The user's query.
        index: The Pinecone index object.
        top_k (int): Number of top relevant documents to retrieve.

    Returns:
        list: A list of relevant documents with metadata and scores.
    """
    try:
        # Generate embedding for the user query
        query_embedding = generate_embedding(user_query)

        # Query Pinecone index
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True  # Retrieve associated metadata
        )

        # Process and return results
        relevant_docs = []
        for match in results.matches:
            relevant_docs.append({
                "id": match.id,
                "score": match.score,
                "metadata": match.metadata
            })

        return relevant_docs

    except Exception as e:
        print(f"An error occurred while querying: {e}")
        return []


In [23]:
# User query
user_query = "Retrieve information about orders and their associated customers."

# Query Pinecone for relevant documents
relevant_docs = query_relevant_documents(user_query, index)

# Print results
for doc in relevant_docs:
    print(f"ID: {doc['id']}, Score: {doc['score']}, Metadata: {doc['metadata']}")


ID: customers, Score: 0.877237499, Metadata: {'columns': ['customerNumber', 'customerName', 'contactLastName', 'contactFirstName', 'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode', 'country', 'salesRepEmployeeNumber', 'creditLimit'], 'description': 'Stores customer information', 'foreign_keys': '{"salesRepEmployeeNumber": "employees.employeeNumber"}', 'primary_key': ['customerNumber'], 'table': 'customers'}
ID: employees, Score: 0.826239645, Metadata: {'columns': ['employeeNumber', 'lastName', 'firstName', 'extension', 'email', 'officeCode', 'reportsTo', 'jobTitle'], 'description': 'Stores employee information', 'foreign_keys': '{"reportsTo": "employees.employeeNumber"}', 'primary_key': ['employeeNumber'], 'table': 'employees'}
ID: offices, Score: 0.797356367, Metadata: {'columns': ['officeCode', 'city', 'phone', 'addressLine1', 'addressLine2', 'state', 'country', 'postalCode', 'territory'], 'description': 'Stores office location information', 'foreign_keys': '{}'